In [3]:
from methods.config import *
from methods.shap_methods import shap_file_path
from methods.pca_methods import PCAExplain
import pickle
import os

d:\Python Projects\Actuarial_NN_Forecasting\test_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# SHAP Values

In [4]:
def display_save_fig(fig, file_name):
    fig.show()
    fig.write_image(os.path.join(IMAGE_PATH, file_name), width=1000, height=500)

In [5]:
VARIABLE = "GS5"  # Select from CPIAUCSL, GS5, RPI and UNRATE

In [6]:
with open(shap_file_path(VARIABLE), "rb") as input_file:
    shap_object = pickle.load(input_file)

In [5]:
fig = shap_object.plot_importance()
display_save_fig(fig, f"shap_importance_{VARIABLE}.png")

In [ ]:
for i in shap_object.importance[::-1][:10].index:
    fig = shap_object.plot_dependence(i)
    display_save_fig(fig, f"shap_dependence_{VARIABLE}_{i}.png")

# PCA

In [ ]:
import re

pcas = {
    int(re.sub("[\(\[].*?[\)\]]", "", i).strip("PCA_"))
    for i in shap_object.importance[::-1][:10].index
    if "PCA" in i
}

In [ ]:
for i in pcas:
    fig = PCAExplain.plot_pca_component(i)
    display_save_fig(fig, f"pca_component_{i}.png")